# Load Testing using Locust

---

모델 배포는 모델 서빙의 첫 단추로 프로덕션 배포 시에 고려할 점들이 많습니다. 예를 들어, 특정 이벤트로 인해 갑자기 동시 접속자가 증가해서 트래픽이 몰릴 수 있죠. SageMaker는 관리형 서비스이니만큼 오토스케일링 policy를 손쉽게 구성할 수 있지만, 비용 최적화 관점에서 최적의 인스턴스 종류와 개수를 정하는 것은 쉽지 않습니다. 따라서, 로드 테스트를 통해 엔드포인트가 처리할 수 있는 RPS(Request Per Second; 동시 초당 접속자)를 파악하는 것이 중요하며, 이를 위해 자체 테스트 툴킷을 개발하거나 오픈소스 툴킷을 사용합니다. (또한, re:Invent 2021에 소개된 신규 서비스인 SageMaker Inference Recommender를 사용하여 로드 테스트를 API 호출로 편리하게 수행할 수 있습니다.)

본 노트북에서는 Locust (https://docs.locust.io/en/stable/) 를 사용하여 간단한 로드 테스트를 수행해 보겠습니다. Locust는 Python으로 테스트 스크립트를 빠르게 작성할 수 있고 파라메터들이 직관적이라 빠르게 로드 테스트 환경을 구축하고 실행할 수 있습니다.

완료 시간은 **10-20분** 정도 소요됩니다. 


### 목차
- [1. Create Locust Script](#1.-Create-Locust-Script)
- [2. Load Testing](#2.-Load-Testing)

<div class="alert alert-warning"><h4>주의</h4><p>
아래 코드 셀은 ngrok 토큰을 설정하고, 주피터 노트북 커널을 셧다운시킵니다. <a href='https://ngrok.com/'>https://ngrok.com/</a> 에서 회원 가입 후, 토큰을 설정해 주시기 바랍니다.
    
노트북 커널이 셧다운된다면, 아래 코드 셀에서 <b><font color='darkred'>setup_needed = False</font></b>로 변경 후, 코드 셀을 다시 실행해 주세요. 이 작업은 한 번만 수행하면 됩니다. 
</p></div>

회원 가입 후, 로그인하면 아래와 같은 화면이 출력됩니다. **2. Connect your account** 에서 `ngrok authtoken [YOUR-TOKEN]`의 `[YOUR-TOKEN]`을 아래 코드 셀로 복사하세요.
![ngrok_1](img/ngrok_1.png)

In [ ]:
import sys, IPython
from pyngrok import ngrok

setup_needed = True
#setup_needed = False

if setup_needed:
    print("===> Setting the authtoken. Please change 'setup_needed = False' and run this code cell again.")
    ngrok.set_auth_token("[YOUR-TOKEN]")
    IPython.Application.instance().kernel.do_shutdown(True)

In [ ]:
%load_ext autoreload
%autoreload 2
%store -r
%store

In [ ]:
try:
    endpoint_name
    s3_path
    test_df
    print("[OK] You can proceed.")
except NameError:
    print("+"*60)
    print("[ERROR] Please run '1_deploy.ipynb' before you continue.")
    print("+"*60)

<br>

# 1. Create Locust Script
---

아래 코드 셀은 Locust 기반 로드 테스트에 필요한 스크립트를 저장합니다. 
- `config.json`: 로드 테스트에서 사용할 설정값들을 저장합니다.
- `stress.py`: 로드 테스트 시 각 사용자의 객체를 생성하는 스크립트로, `HttpUser` 클래스를 상속받습니다. 이 클래스는 각 사용자에게 client 속성을 부여합니다. 

In [ ]:
%%writefile config.json
{
    "contentType": "text/csv",
    "showEndpointResponse": 0,
    "dataFile": "../data/dataset/test.csv",
    "numTestSamples": 100
}

In [ ]:
%%writefile stress.py
import os
import json
import time
import boto3
import io
from io import StringIO
import pandas as pd
from locust import HttpUser, task, events, between


class SageMakerConfig:

    def __init__(self):
        self.__config__ = None

    @property
    def data_file(self):
        return self.config["dataFile"]

    @property
    def content_type(self):
        return self.config["contentType"]

    @property
    def show_endpoint_response(self):
        return self.config["showEndpointResponse"]
    
    @property
    def num_test_samples(self):
        return self.config["numTestSamples"]

    @property
    def config(self):
        self.__config__ = self.__config__ or self.load_config()
        return self.__config__

    def load_config(self):
        config_file = os.path.join(os.path.dirname(os.path.realpath(__file__)), "config.json")
        with open(config_file, "r") as c:
            return json.loads(c.read())


    
class SageMakerEndpointTestSet(HttpUser):
    wait_time = between(5, 15)
    
    def __init__(self, parent):
        super().__init__(parent)
        self.config = SageMakerConfig()
        
        
    def on_start(self):
        data_file_full_path = os.path.join(os.path.dirname(__file__), self.config.data_file)
        
        test_df = pd.read_csv(data_file_full_path)
        y_test = test_df.iloc[:, 0].astype('int')
        test_df = test_df.drop('fraud', axis=1)

        csv_file = io.StringIO()
        test_df[0:self.config.num_test_samples].to_csv(csv_file, sep=",", header=False, index=False)
        self.payload = csv_file.getvalue()
        

    @task
    def test_invoke(self):
        response = self._locust_wrapper(self._invoke_endpoint, self.payload)
        if self.config.show_endpoint_response:
            print(response["Body"].read())

    
    def _invoke_endpoint(self, payload):
        region = self.client.base_url.split("://")[1].split(".")[2]
        endpoint_name = self.client.base_url.split("/")[-2]
        runtime_client = boto3.client('sagemaker-runtime', region_name=region)

        response = runtime_client.invoke_endpoint(
            EndpointName=endpoint_name,
            Body=payload,
            ContentType=self.config.content_type
        )

        return response
    

    def _locust_wrapper(self, func, *args, **kwargs):
        """
        Locust wrapper so that the func fires the sucess and failure events for custom boto3 client
        :param func: The function to invoke
        :param args: args to use
        :param kwargs:
        :return:
        """
        start_time = time.time()
        try:
            result = func(*args, **kwargs)
            total_time = int((time.time() - start_time) * 1000)
            events.request.fire(request_type="boto3", 
                                name="invoke_endpoint", 
                                response_time=total_time, 
                                response_length=0)

            return result
        except Exception as e:
            total_time = int((time.time() - start_time) * 1000)
            events.request.fire(request_type="boto3", 
                                name="invoke_endpoint", 
                                response_time=total_time, 
                                response_length=0,
                                exception=e)

            raise e

<br>

# 2. Load Testing
---

로드 테스트는 아래 파라메터들의 설정만으로 로드 테스트를 편리하게 수행할 수 있습니다.

- `num_users`: 어플리케이션을 테스트하는 총 사용자 수입니다. 
- `spawn_rate`: 초당 몇 명씩 사용자를 늘릴 것인지 정합니다. 이 때, on_start 함수가 정의되어 있다면 이 함수를 같이 호출합니다.

예를 들어 `num_users=100, spawn_rate=10` 일 때는 초당 10명의 사용자가 추가되며, 10초 후에는 100명의 사용자로 늘어납니다. 이 사용자 수에 도달하면 통계치가 재설정되니다.

In [ ]:
import boto3
region = boto3.Session().region_name

num_users = 100
spawn_rate = 10
endpoint_url = f'https://runtime.sagemaker.{region}.amazonaws.com/endpoints/{endpoint_name}/invocations'

### Running a locustfile

주피터 노트북 상에서의 실습을 위해 nohup으로 백그라운드에서 locust를 시작합니다. Locust는 기본적으로 8089 포트를 사용합니다. (http://localahost:8089)

In [ ]:
# %%bash -s "$num_users" "$spawn_rate" "$endpoint_url"

# echo locust -f stress.py -u $1 -r $2 -H $3

In [ ]:
%%bash -s "$num_users" "$spawn_rate" "$endpoint_url"

nohup locust -f stress.py -u $1 -r $2 -H $3 >/dev/null 2>&1 &

### Secure tunnels to localhost using ngrok

ngrok를 사용해 외부에서 로컬호스트로 접속할 수 있습니다. pyngrok는 Python wrapper로 API 호출로 ngrok을 더 편리하게 사용할 수 있습니다.

- ngrok: https://ngrok.com/
- pyngrok: https://pyngrok.readthedocs.io/en/latest

In [ ]:
http_tunnel = ngrok.connect(8089, bind_tls=True)
http_url = http_tunnel.public_url

아래 코드 셀 실행 시 출력되는 URL을 클릭 후, `Start swarming` 버튼을 클릭해 주세요.

In [ ]:
from IPython.core.display import display, HTML
display(HTML(f'<b><a target="blank" href="{http_url}">Load test: {http_url}</a></b>'))

![locust_1](img/locust_1.png)
![locust_2](img/locust_2.png)

In [ ]:
tunnels = ngrok.get_tunnels()
print(tunnels)

### CloudWatch Monitoring
아래 코드 셀에서 출력되는 링크를 클릭해면 CloudWatch 대시보드로 이동합니다.

In [ ]:
cw_url = f"https://console.aws.amazon.com/cloudwatch/home?region={region}#metricsV2:graph=~(metrics~(~(~'AWS*2fSageMaker~'InvocationsPerInstance~'EndpointName~'{endpoint_name}~'VariantName~'AllTraffic))~view~'timeSeries~stacked~false~region~'{region}~start~'-PT15M~end~'P0D~stat~'SampleCount~period~60);query=~'*7bAWS*2fSageMaker*2cEndpointName*2cVariantName*7d*20{endpoint_name}"
display(HTML(f'<b><a target="blank" href="{cw_url}">Cloudwatch Monitoring</a></b>'))

### Stop Locust and Disconnect ngrok

In [ ]:
!pkill -9 -ef locust
ngrok.disconnect(http_url)

### (Optional) More testing

위 섹션에서 `num_users, spawn_rate`를 변경해서 테스트해 보세요. (예: `num_users=1000, spawn_rate=20`) RPS가 일정 이상이면 Failures 수치가 올라가는 것을 확인할 수 있습니다.

<br>

# 3. Endpoint Clean-up
---

과금 방지를 위해 엔드포인트를 삭제합니다.

In [ ]:
import sagemaker
from sagemaker.predictor import Predictor
from sagemaker.serializers import CSVSerializer

sess = sagemaker.Session()
xgb_predictor = Predictor(
    endpoint_name=endpoint_name, 
    sagemaker_session=sess,
    serializer=CSVSerializer()
)

In [ ]:
xgb_predictor.delete_endpoint()